## Kravspecifikation för del 2

1. ✅ Börja använda basala **MLOps best-practices:**
    1. 📊 Ordna så att du har ett bra sätt att övervaka träningen, samt att spara (och hålla ordning på!) versioner av modellerna du tränar upp. (Tips: Foldrar, smart valda filnamn, etc).
    2. 💾 Kom ihåg att det troligen inte är den sista epokens checkpoint (sparad version) som är den bästa, p.g.a. overfitting om du kör många epochs. Du behöver själv se till att spara checkpoints lite då och då under träningen.
    3. 🗄️ Versionshantering av parametrar, kod, och checkpoints (modellens parametrar) blir snabbt väldigt viktigt när man börjar jobba seriöst med machine learning. Om man inte håller reda på vilken körning som gjorts med vilka parametrar, så blir det svårt att jämföra prestanda, vidareutveckla modellen senare, etc. Ingen vill ha en modell levererad där programmeraren inte minns hur den gjordes.
    4. 🚀 **Fördjupning/överkurs:** Vilka verktyg finns som kan hjälpa till att hålla ordning på versioner, körningar, resultat, hyper parameters, etc?
2. 🚀 **Fördjupning/överkurs:** Addera **performance metrics**, så att du kan se hur lång tid respektive tränings-körning tar. (En körning = Alla epochs av träning för en given modell med specificerad uppsättning hyper parameters).
3. ✅ Applicera lämpliga **data augmentation** methods för att artificiellt variera, och till och med “förstora” datamängden lite. Använd exempelvis skalning, rotation, färgvariation, brus, (spegelvändning?), etc. Här är lite repetition om data augmentation:
    1. Se avsnittet “[Uppgift 1 Perceptron för OCR](https://www.notion.so/Uppgift-1-Perceptron-fo-r-OCR-d9e945f1551341e88665775acacc0bb6?pvs=21)” längre ner i denna uppgift (del 3).
    2. Bra 10-minuters video om ämnet: [Pytorch-Data-Augmentation-using-Torchvision](https://youtu.be/Zvd276j9sZ8?si=9fGaNnoRzsHrvxB6)
    3. Observera att det inte är säkert att data augmentation leder till bättre resultat i alla lägen, eftersom det beror på detaljerna i vad man bygger, och vilken data man har.
4. ✅ Det är nu dags att testa CNN istället för FFN. Byt ut de första lagren i modellen till **convolutional layers** för att ta upp translationsinvarians i bilden. Sök själv upp vad man brukar ha ***direkt efter varje convolutional layer*** för att hålla ordning på antalet dimensioner till nästa lager .
    
    > (Svårt ord “translations-invarians”: translation=förflyttning, invarians=”samma oavsett” ⇒ translationsinvarians = ”spelar ingen roll var i bilden”)
    > 
5. ✅ Experimentera därefter med att addera ett till (eller flera) convolutional layer(s). Tanken är att testa **olika nät-arkitekturer**. Använd Google/Chat för initial gissning för modell-arkitektur, men sedan är det trial-and-error som gäller. Att fundera på:
    1. ⚖️ Hur påverkas resultatet av olika modell-arkitektur? Var noga med att inte jämföra modeller med stor skillnad i antal parameters, om aspekter som tex lagertyp ska utvärderas.
    2. 🤔 Vilken typ av features detekteras typiskt av de senare convolutional-lagren jämfört med det första convolutional-lagret?
        1. Hint: Videon från lektion om convolutional layers: [Convolutional-Neural-Networks-Explained](https://youtu.be/pj9-rr1wDhM?si=cjWmR5ets048WjfZ). (Självfallet kan du också plotta weight-matrices från din egen modell också om du vill!)
6. ✅ Applicera lämpliga **regularization methods** för att minimera risken för overfitting och göra cost-function-landskapet fördelaktigt för back-prop.
    1. Exempel på sådana metoder är: drop-out, weight-decay, noise injection, batch normalization, etc.
    2. Mer info om regularization: [understanding-regularization-with-pytorch](https://medium.com/analytics-vidhya/understanding-regularization-with-pytorch-26a838d94058)
7. 🚀 **Fördjupning:** Skapa en lista av kombinationer av hyper parameters, och låt datorn träna din modell på nytt för samtliga hyperparameter settings i din lista. Detta kallas för **hyper-parameter tuning**, d.v.s. att via trial-and-error hitta inställningar som funkar.
    - Enklast möjliga hyper-param-tuning: Gör en lista med t.ex. 10 rader (en per körning), där varje rad specificerar alla hyper-parameters som kan vara svårt att gissa optimalt värde på:
        - Antal neuroner för respektive lager
        - Typ av activation function
        - Vilka lager som ska vara convolutional, storlek på convolutional kernel, antal convolutional kernels per lager, etc.
        - Learning rate / hyper-params till optimizer ADAM
        - Drop-out level
        - Settings för olika typer av data augmentation (rotation, skalning, noise, etc)
    - Det finns även en mängd automatiserade typer av hyper-param-tuning. Se exempelvis: https://en.wikipedia.org/wiki/Hyperparameter_optimization

In [2]:
# Define the perceptron neural-network model

import torch
import torch.nn as nn

class Perceptron(nn.Module):
    # Define the constructor
    def __init__(self):
        super().__init__()

        # Flatten the input
        self.flatten = nn.Flatten()

        # Define the layers with ReLU activation function
        self.linear_relu_stack = nn.Sequential(
            # Input layer   
            nn.Linear(28*28, 512),
            nn.ReLU(),

            # Hidden layer
            nn.Linear(512, 512),
            nn.ReLU(),

            # Output layer
            nn.Linear(512, 10),
        )

    # Define the forward pass
    def forward(self, x):
        # Flatten the input
        x = self.flatten(x)

        # Pass through the layers
        logits = self.linear_relu_stack(x)
        return logits

In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device\n")

model = Perceptron().to(device)
print(model)

Using cuda device

Perceptron(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
# Download MNIST dataset and create dataloaders

from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

# Transformations for training data as data augmentation
train_transform = transforms.Compose([
    transforms.RandomRotation(10),  # Rotate by up to 10 degrees
    transforms.RandomAffine(0, scale=(0.8, 1.2)),  # Random scaling
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.1307,), (0.3081,)),  # Normalize with MNIST mean/std (pre-computed)
])

train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=train_transform
)

# Transformations for test data (no augmentation)
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=test_transform
)

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

In [7]:
# Train the network using ADAM back-propagation

import os
import logging
import json
import time
import torch.optim as optim
from datetime import datetime

# Generate a unique run ID for this experiment and create a directory for it
os.makedirs('models', exist_ok=True)
run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
run_dir = os.path.join('models', f'run_{run_id}')
os.makedirs(run_dir, exist_ok=True)

# Configure logging and create a logger object and log file
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
log_file = os.path.join(run_dir, 'training.log')
fhandler = logging.FileHandler(filename=log_file, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

# Define hyperparameters
hyperparams = {
    'learning_rate': 0.001,
    'num_epochs': 5,
}

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=hyperparams['learning_rate'])

# Save hyperparameters for versioning
with open(os.path.join(run_dir, 'hyperparams.json'), 'w') as f:
    json.dump(hyperparams, f, indent=4)

# Training loop
def train():
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        
        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(pred.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            logger.info(f"    [{current:>5d}/{len(train_dataloader.dataset):>5d}] - Loss: {loss:>7f}")

    return train_loss / len(train_dataloader)

# Test loop
def test():
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()
            _, predicted = torch.max(pred.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
    
    test_loss /= len(test_dataloader)
    test_accuracy = 100 * correct / total

    logger.info(f"    Test accuracy: {test_accuracy}")
    logger.info(f"    Test avg loss: {test_loss}")

    return test_loss, test_accuracy

# Save model checkpoint
def save_checkpoint(epoch, file_path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'hyperparams': hyperparams
    }
    
    # Save checkpoint
    torch.save(checkpoint, file_path)
    logger.info(f"    Checkpoint saved to {file_path}")

# Train the model
total_start_time = time.time()
best_accuracy = 0
best_epoch = 1

logger.info("-" * 80)
for epoch in range(hyperparams['num_epochs']):
    logger.info(f"Epoch {epoch+1}:")

    epoch_start_time = time.time()

    # Training phase
    train_start_time = time.time()
    train_loss = train()
    train_elapsed_time = time.time() - train_start_time

    # Testing phase
    test_start_time = time.time()
    test_loss, test_accuracy = test()
    test_elapsed_time = time.time() - test_start_time

    epoch_elapsed_time = time.time() - epoch_start_time

    # Update best accuracy and epoch
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_epoch = epoch + 1

    # Save checkpoint to file
    save_checkpoint(epoch, os.path.join(run_dir, f'checkpoint_epoch_{epoch+1}.pt'))

    # Log epoch results
    logger.info(f"    Training time: {train_elapsed_time:.2f} seconds")
    logger.info(f"    Testing time: {test_elapsed_time:.2f} seconds")
    logger.info(f"    Total epoch time: {epoch_elapsed_time:.2f} seconds")
    logger.info("-" * 80)

# Log total training time and best epoch
total_elapsed_time = time.time() - total_start_time
logger.info("Training complete!")
logger.info(f"Total training time: {total_elapsed_time:.2f} seconds")
logger.info(f"Best epoch: {best_epoch}")
logger.info("-" * 80)

2025-04-16 20:08:13,906 - INFO - --------------------------------------------------------------------------------
2025-04-16 20:08:13,907 - INFO - Epoch 1:
2025-04-16 20:08:13,937 - INFO -     [   64/60000] - Loss: 2.961130
2025-04-16 20:08:15,368 - INFO -     [ 6464/60000] - Loss: 0.207177
2025-04-16 20:08:16,815 - INFO -     [12864/60000] - Loss: 0.249086
2025-04-16 20:08:18,262 - INFO -     [19264/60000] - Loss: 0.064662
2025-04-16 20:08:19,670 - INFO -     [25664/60000] - Loss: 0.186498
2025-04-16 20:08:21,064 - INFO -     [32064/60000] - Loss: 0.248541
2025-04-16 20:08:22,472 - INFO -     [38464/60000] - Loss: 0.069734
2025-04-16 20:08:23,883 - INFO -     [44864/60000] - Loss: 0.031699
2025-04-16 20:08:25,285 - INFO -     [51264/60000] - Loss: 0.080397
2025-04-16 20:08:26,678 - INFO -     [57664/60000] - Loss: 0.076674
2025-04-16 20:08:28,390 - INFO -     Test accuracy: 98.33
2025-04-16 20:08:28,390 - INFO -     Test avg loss: 0.053572002546301935
2025-04-16 20:08:28,400 - INFO - 